### background
I have a list of N people who need to be seated in a boat in N seats. 
Boat has one cox, and 7 pairs of rowers. N = 15.
People need to be matched by weight, such that the boat is balanced in terms of roll and pitch (https://commons.wikimedia.org/wiki/File:Rotations.png#/media/File:Rotations.png)
and by height, so that people have neighbours of similar heights (to make rowing as a unit easier).
People also have preferences:
* To cox or not to cox. 
* Sit in the front/back/middle of the boat. NB: height makes it uncomfortable for tall people to sit in the first or last row.
* In case of switching, some people would like to switch sides.

Switching.
in total there are 4 seating plans to make, since seating will be switched 4 times throughout the race (see last point in preferences).

### solution
it can be seen as a variant of a stable-marriage problem (https://en.wikipedia.org/wiki/Stable_marriage_problem). SMP is the problem of finding a stable matching between two equally sized sets of elements given an ordering of preferences for each element. 
There are two differences: 
* it is unidirectional (so rowers *prefer* sits, but sits *do not prefer* rowers).
* it is with indifference, so it needs not to exist a strict order of preferences, e.g. top 3 sits are given, rest is of equal priority. see https://en.wikipedia.org/wiki/Stable_marriage_with_indifference

the assignment problem might be good to look into, if we are able to translate weight/height matches into some edge weights. The assignment problem consists of finding, in a weighted bipartite graph, a matching in which the sum of weights of the edges is as large as possible. A common variant consists of finding a minimum-weight perfect matching. see https://en.wikipedia.org/wiki/Assignment_problem

#### for NetPharMed:
There is an implementation of a classical bidirectional aglo implemented as Matcher() below. It matches rowers with seats. From the point of view of a rower, it is truthful, ie no better seating can be achieved by misinforming the preferences. To adapt it for office seating, make all the sits have identical preferences for rowers. That can also be used in the git manual, to make people branch the file, input their preferences, and then submit a pull request.

#### for Sulkava:
how to make matching by weight/height happen? One idea is to use simulated annealing. SA is an optimization technique to find a global optimum of a function, esp when the search space is discrete. For problems where finding an *approximate global optimum* is more important than finding a *precise local optimum in a fixed amount of time*, simulated annealing may be preferable to alternatives such as gradient descent. The notion of slow cooling (from metallurgy) implemented in the simulated annealing algorithm is interpreted as a slow decrease in the probability of accepting worse solutions as the solution space is explored. At each time step, the algorithm randomly selects a solution close to the current one, measures its quality, and then decides to move to it or to stay with the current solution based on either one of two probabilities between which it chooses on the basis of the fact that the new solution is better or worse than the current one. During the search, the temperature is progressively decreased from an initial positive value to zero and affects the two probabilities: at each step, the probability of moving to a better new solution is either kept to 1 or is changed towards a positive value; on the other hand, the probability of moving to a worse new solution is progressively changed towards zero. Here it is in gif form: https://commons.wikimedia.org/wiki/File:Hill_Climbing_with_Simulated_Annealing.gif#/media/File:Hill_Climbing_with_Simulated_Annealing.gif

Here is the implementation of SA for wedding table seating (where there are N tables for P people, each table sits M people, match people's preferences so that they do not sit with those they hate, and tend to preserve their groups of friends. https://lukeplant.me.uk/blog/posts/wedding-hacks---seating-planner-using-simulated-annealing/ The idea is that you define a matrix of connections, with a zero indicating that the people don’t know each other, 1 that they do know each other, plus 50 indicating that they must be together, and negative numbers for people who should be kept apart!
Here is an alternative to it: http://linanqiu.github.io/2018/03/05/Wedding-Seat-Optimization/ with a Jup notebook https://github.com/linanqiu/wedding-optimization-simulated-annealing/blob/master/wedding-optimization.ipynb
and another https://github.com/zneedell/seating_optimization/blob/master/main.ipynb

Here is a different solution of the wedding seat solved using hypergraphs. We model it as a hypergraph partitioning problem and solving it with KaHyPar (http://kahypar.org/ / http://github.com/SebastianSchlag/kahypar). Every guest is represented by a node and weighted hyperedges express relationships (a couple and their kids, extended family, friends, acquaintances, etc). The goal is to partition the hypergraph into (roughly) equal-sized parts (i.e., tables) while minimising the sum of weights of cut hyperedges ("λ-1 metric").

This is a solution using simple linear programming https://stackoverflow.com/questions/19143474/which-algorithm-could-solve-my-wedding-table-issue but without code.

In [118]:
# refer to gale-shapley 1962 paper for the original algo

from collections import defaultdict


class Matcher:

    def __init__(self, rowers, sits, forbidden):
        '''
        Constructs a Matcher instance.
        Takes a dict of rowers's sittings preferences, `rowers`,
        a dict of special sits to be allocated e.g. white oars, `sits`,
        and a dict specifying which sittings are forbidden
        for each rower:
        >>> forbidden = { 'name': ['3', '4', ... ] }
        '''
        self.R = rowers
        self.S = sits
        self.forbidden = forbidden
        self.sittings = {}
        self.pairs = []

        # we rank sitting preferences at initialization 
        # to avoid expensive lookups when matching
        self.rrank = defaultdict(dict)  # `rrank[r][s]` is r's ranking of s
        self.srank = defaultdict(dict)  # `srank[s][r]` is s's ranking of r

        for r, prefs in rowers.items():
            for i, s in enumerate(prefs):
                self.rrank[r][s] = i

        for s, prefs in sits.items():
            for i, r in enumerate(prefs):
                self.srank[s][r] = i

    def __call__(self):
        return self.match()

    def prefers(self, s, r, h):
        '''
        Test whether s prefers r over h.
        '''
        return self.srank[s][r] < self.srank[s][h]

    def is_forbidden(self, r, s):
        '''
        Test whether (r, s) is a forbidden pairing.
        '''
        return s in self.forbidden.get(r, [])

    def after(self, r, s):
        '''
        Return the sit favored by r after s.
        
        '''
        prefs = self.R[r]               # r's ordered list of preferences
        i = self.rrank[r][s] + 1        # index of sits following s in list of prefs
        if i >= len(prefs):
            return ''                   # no more sits left!
        s = prefs[i]                    # sit following s in list of prefs
        if self.is_forbidden(r, s):     # if r, s) is forbidden
            return self.after(r, s)     # try next w 
        return s

    def match(self, rowers=None, next=None, sittings=None):
        '''
        Try to match all rowers with their next preferred sit.
        
        '''
        if rowers is None: 
            rowers = list(self.R)         # get the complete list of rowers
        if next is None: 
            # if not defined, map each rower to their first preference
            next = dict((r, rank[0]) for r, rank in self.R.items()) 
        if sittings is None: 
            sittings = {}                  # mapping from sit to current sitting
        if not len(rowers): 
            self.pairs = [(h, s) for s, h in sittings.items()]
            self.sittings = sittings
            return sittings
        r, rowers = rowers[0], rowers[1:]
        s = next[r]                     # next sit for m to choose
        if not s:                       # continue if no sit to choose
            return self.match(rowers, next, sittings)
        next[r] = self.after(r, s)      # sit after s in r's list of prefs
        if s in sittings:
            h = sittings[s]                # current sitting
            if self.prefers(s, r, h):
                rowers.append(h)           # sit becomes available again
                sittings[s] = r            # s becomes sit for r
            else:
                rowers.append(r)           # r remains unsitted
        else:
            sittings[s] = r                # s becomes s for r
        return self.match(rowers, next, sittings)

    def is_stable(self, sittings=None, verbose=False):
        if sittings is None:
            sittings = self.sittings
        for s, r in sittings.items():
            i = self.R[r].index(s)
            preferred = self.R[r][:i]
            for p in preferred:
                if p in self.forbidden.get(r, []):  # no need to worry about
                    continue                        # forbidden configs
                if not p in sittings:
                    continue
                h = sittings[p]
                if self.S[p].index(r) < self.S[p].index(h):  
                    msg = "{}'s sitting to {} is unstable: " + \
                          "{} prefers {} over {} and {} prefers " + \
                          "{} over the current sitting {}"
                    if verbose:
                        print(msg.format(r, s, r, p, s, p, r, h))
                    return False
        return True


In [119]:
# rowers is just people with their sit preferences
rowers = "tiia: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; alison: 1, 2, 3, 4, 5, 10, 11, 12, 13, 6, 7, 8, 9, 14, 15; bulat: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; arvydas: 1, 2, 3, 13, 14, 15, 4, 5, 6, 7, 8, 9, 10, 11, 12; julia: 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5; lauri: 1, 2, 10, 11, 12, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15; tatiana: 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 14, 15; ulrika: 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 14, 15; karen: 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10; hans: 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15; jaana: 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15; joseph: 1, 2, 11, 12, 13, 14, 15, 3, 4, 5, 6, 7, 8, 9, 10; elli: 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8; elina: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; kul: 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11"
                                    
# sits is                                     
sits = "1: tiia, alison, bulat, arvydas, julia, lauri, tatiana, ulrika, karen, hans, jaana, joseph, elli, elina, kul; 2: tiia, alison, julia, lauri, bulat, arvydas, tatiana, ulrika, karen, hans, jaana, joseph, elli, elina, kul; 3: lauri, bulat, arvydas, tatiana, ulrika, tiia, alison, julia, karen, hans, jaana, joseph, elli, elina, kul; 4: jaana, joseph, elli, elina, kul, lauri, bulat, arvydas, tatiana, ulrika, tiia, alison, julia, karen, hans; 5: arvydas, tatiana, ulrika, tiia, alison, julia, karen, hans, jaana, joseph, elli, elina, kul, lauri, bulat; 6: arvydas, alison, julia, karen, hans, jaana, tatiana, ulrika, tiia, joseph, elli, elina, kul, lauri, bulat; 7: ulrika, tiia, joseph, elli, elina, kul, lauri, bulat, arvydas, alison, julia, karen, hans, jaana, tatiana; 8: lauri, bulat, arvydas, ulrika, tiia, joseph, elli, elina, kul, alison, julia, karen, hans, jaana, tatiana; 9: elina, kul, alison, julia, karen, lauri, bulat, arvydas, ulrika, tiia, joseph, elli, hans, jaana, tatiana; 10: elina, kul, alison, julia, karen, lauri, hans, jaana, tatiana, bulat, arvydas, ulrika, tiia, joseph, elli; 11: tatiana, bulat, arvydas, ulrika, tiia, joseph, elli, elina, kul, alison, julia, karen, lauri, hans, jaana; 12: tatiana, bulat, arvydas, elli, elina, kul, ulrika, tiia, joseph, alison, julia, karen, lauri, hans, jaana; 13: bulat, arvydas, alison, julia, karen, hans, jaana, tatiana, ulrika, tiia, joseph, elli, elina, kul, lauri; 14: joseph, elli, elina, kul, lauri, bulat, arvydas, alison, julia, karen, hans, jaana, tatiana, ulrika, tiia; 15: joseph, kul, elli, elina, lauri, bulat, arvydas, alison, julia, karen, hans, jaana, tatiana, ulrika, tiia"

forbidden = "1: jaana, joseph; 2: joseph, elli; 3: elli, elina; 4: elina, kul; 5: kul, tiia; 6: tiia, alison; 7: alison, bulat; 8: bulat, arvydas; 9: arvydas, julia; 10: julia, lauri; 11: lauri, tatiana; 12: tatiana, ulrika; 13: ulrika, karen; 14: karen, hans; 15: hans, jaana"


In [120]:
def dictor(s):
    '''
    returns a dict from a string. Given in the following format
    name0: preference0, preferenceN; nameK: preference0, preferenceN;
    '''
    return dict((k.strip(), v.strip().replace(',','').split()) for k,v in (item.split(':') for item in s.split(';')))

In [121]:
r = dictor(rowers)
s = dictor(sits)
f = dictor(forbidden)

In [122]:
out = Matcher(r, s, f)

In [123]:
out()

{'1': 'tiia',
 '6': 'julia',
 '5': 'tatiana',
 '7': 'ulrika',
 '11': 'joseph',
 '9': 'elli',
 '12': 'kul',
 '2': 'alison',
 '10': 'lauri',
 '3': 'bulat',
 '13': 'arvydas',
 '4': 'jaana',
 '14': 'karen',
 '8': 'elina',
 '15': 'hans'}